In [1]:
!pip install geopandas
import geopandas as gpd
import pandas as pd
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()

     |████████████████████████████████| 1.0 MB 15.3 MB/s 
     |████████████████████████████████| 16.7 MB 255 kB/s 
     |████████████████████████████████| 6.3 MB 29.6 MB/s 


DataTransformerRegistry.enable('default')

In [10]:
mmg = pd.read_csv('https://raw.githubusercontent.com/waltsco059/gis_repo/main/texas_mmg.csv')
mmg.head()

,FIPS,State,County,FIR,FIP,Lo_Thresh,Lo_Type,Hi_Thresh,Hi_Type,P_Below,...,Budget,Year,Total,White,Black,Native,Asian,Pacific,Other,Biracial
0,48001,TX,Anderson,0.210,12210,1.65,SNAP,1.85,Other Nutrition Program,0.482,...,4981040,2010,57922.0,0.602914,0.199872,0.006163,0.006768,0.000224,0.121992,0.062066
1,48003,TX,Andrews,0.147,2060,1.65,SNAP,1.85,Other Nutrition Program,0.619,...,946750,2010,18610.0,0.586190,0.012735,0.011822,0.006018,0.000000,0.176142,0.207093
2,48005,TX,Angelina,0.194,16570,1.65,SNAP,1.85,Other Nutrition Program,0.637,...,6360380,2010,86395.0,0.635303,0.151606,0.006517,0.010637,0.000486,0.099404,0.096047
3,48007,TX,Aransas,0.174,4050,1.65,SNAP,1.85,Other Nutrition Program,0.637,...,1582470,2010,23830.0,0.755183,0.011078,0.010281,0.021024,0.000713,0.065338,0.136383
4,48009,TX,Archer,0.149,1350,1.65,SNAP,1.85,Other Nutrition Program,0.457,...,613470,2010,8560.0,0.882126,0.003738,0.012617,0.002220,0.000000,0.035164,0.064136


In [4]:
# process county shapefile
shapefile = gpd.read_file("https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json")
shapefile = shapefile.rename(columns={'id': 'FIPS'}).astype({"FIPS": int})

# create mini dataframes for manipulaiton
map = mmg[['FIPS', 'FIR', 'Year']]
county_names = mmg[['FIPS', 'County']].drop_duplicates()
demographics = mmg[['FIPS', 'County', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Other', 'Biracial']].drop_duplicates()

# pivot the map df for the year slider
map['Year'] = map['Year'].astype(str)
map = map.pivot(index='FIPS', columns='Year', values='FIR').reset_index()

columns = [str(Year) for Year in range(2010, 2020)]

# this step ensures the fir_map will have county names & total info 
shapefile = shapefile.merge(county_names, left_on='FIPS', right_on='FIPS')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [5]:
demographics

,FIPS,County,White,Black,Native,Asian,Pacific,Other,Biracial
0,48001,Anderson,0.602914,0.199872,0.006163,0.006768,0.000224,0.121992,0.062066
1,48003,Andrews,0.586190,0.012735,0.011822,0.006018,0.000000,0.176142,0.207093
2,48005,Angelina,0.635303,0.151606,0.006517,0.010637,0.000486,0.099404,0.096047
3,48007,Aransas,0.755183,0.011078,0.010281,0.021024,0.000713,0.065338,0.136383
4,48009,Archer,0.882126,0.003738,0.012617,0.002220,0.000000,0.035164,0.064136
...,...,...,...,...,...,...,...,...,...
249,48499,Wood,0.822224,0.037754,0.007560,0.005129,0.000089,0.048592,0.078652
250,48501,Yoakum,0.488433,0.008188,0.008058,0.006499,0.000000,0.255134,0.233689
251,48503,Young,0.805843,0.010242,0.010298,0.004981,0.000000,0.084625,0.084010
252,48505,Zapata,0.390309,0.001296,0.004824,0.001152,0.000000,0.147455,0.454964


In [6]:
mmg['FIP_Budget'] = mmg['Budget'] / mmg['FIP']
mmg

,FIPS,State,County,FIR,FIP,Lo_Thresh,Lo_Type,Hi_Thresh,Hi_Type,P_Below,...,Year,Total,White,Black,Native,Asian,Pacific,Other,Biracial,FIP_Budget
0,48001,TX,Anderson,0.210,12210,1.65,SNAP,1.85,Other Nutrition Program,0.482,...,2010,57922.0,0.602914,0.199872,0.006163,0.006768,0.000224,0.121992,0.062066,407.947584
1,48003,TX,Andrews,0.147,2060,1.65,SNAP,1.85,Other Nutrition Program,0.619,...,2010,18610.0,0.586190,0.012735,0.011822,0.006018,0.000000,0.176142,0.207093,459.587379
2,48005,TX,Angelina,0.194,16570,1.65,SNAP,1.85,Other Nutrition Program,0.637,...,2010,86395.0,0.635303,0.151606,0.006517,0.010637,0.000486,0.099404,0.096047,383.849125
3,48007,TX,Aransas,0.174,4050,1.65,SNAP,1.85,Other Nutrition Program,0.637,...,2010,23830.0,0.755183,0.011078,0.010281,0.021024,0.000713,0.065338,0.136383,390.733333
4,48009,TX,Archer,0.149,1350,1.65,SNAP,1.85,Other Nutrition Program,0.457,...,2010,8560.0,0.882126,0.003738,0.012617,0.002220,0.000000,0.035164,0.064136,454.422222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,48499,TX,Wood,0.169,7520,1.65,SNAP,1.85,Other Nutrition Program,0.529,...,2019,44843.0,0.822224,0.037754,0.007560,0.005129,0.000089,0.048592,0.078652,531.781915
2536,48501,TX,Yoakum,0.090,780,1.65,SNAP,1.85,Other Nutrition Program,0.820,...,2019,7694.0,0.488433,0.008188,0.008058,0.006499,0.000000,0.255134,0.233689,530.769231
2537,48503,TX,Young,0.143,2570,1.65,SNAP,1.85,Other Nutrition Program,0.587,...,2019,17867.0,0.805843,0.010242,0.010298,0.004981,0.000000,0.084625,0.084010,513.229572
2538,48505,TX,Zapata,0.194,2780,1.65,SNAP,1.85,Other Nutrition Program,0.915,...,2019,13889.0,0.390309,0.001296,0.004824,0.001152,0.000000,0.147455,0.454964,419.424460


In [7]:
counties = shapefile
source = map

# Interactions
slider = alt.binding_range(min=2010, max=2019, step=1, name='Year')
select_year = alt.selection_single(fields=['Year'], bind=slider, init={'Year': 2010})
select_region = alt.selection_multi(on='click', fields=['FIPS'])

# Food Insecurity Map
fir_map = alt.Chart(counties).mark_geoshape(stroke='#fff').encode(
    color=alt.Color('FIR:Q', legend=alt.Legend(format=".0%"), scale=alt.Scale(scheme='purplebluegreen')),
    opacity=alt.condition(select_region, alt.value(1), alt.value(0.1)),
    tooltip = ['County:N', alt.Tooltip('FIR:Q', format='.1%'), 'Year:Q']
).project(
    type='albersUsa'
).transform_lookup(
    lookup='FIPS',
    from_=alt.LookupData(source, 'FIPS', columns)
).transform_fold(
    columns, as_=['Year', 'FIR']
).transform_calculate(
    Year ='parseInt(datum.Year)' 
).transform_filter(
    select_year
).add_selection(
    select_year,
    select_region
).properties(
    width=800,
    height=300,
    title="Texas County Food Insecurity Rate (2010-2019)"
)

# Demographic Pie Chart
pie_demographics = alt.Chart(demographics).mark_arc(innerRadius=100, width=2).encode(
    theta=alt.Theta(field='Percent', type='quantitative'),
    color=alt.Color(field='Race', type='nominal'),
    tooltip=['County:N', 'Race:N', alt.Tooltip('Percent:Q', format='.1%')]
).transform_fold(
    ['White', 'Black', 'Native', 'Asian', 'Pacific', 'Other', 'Biracial'], as_=['Race', 'Percent']
).add_selection(
    select_region,
).transform_filter(
    select_region
).properties(
    title="County Demographics"
)

# Budget Line Chart
budget_line = alt.Chart(mmg).mark_line(point=True).encode(
    x='Year:N',
    y= alt.Y('FIP_Budget:Q', axis=alt.Axis(format='$,f', title='Budget ($)')),
    color='County:N',
    tooltip = ['County:N', alt.Tooltip('FIP_Budget:Q', format='$,.2f', title='Budget'), 'Year:Q']
).add_selection(
    select_region,
).transform_filter(
    select_region
).properties(
    title="Budget Needed per Food Insecure Person"
)

In [8]:
food_security = (fir_map & (pie_demographics | budget_line).resolve_scale(color='independent'))

In [9]:
food_security

alt.VConcatChart(...)

In [ ]:
food_security.save('food_security.html')